# Identifying Regions of Interest with Segmentation

### Environment Setup
⚠️**Note: If running on a new environment, run this cell once and then restart the kernel**⚠️

In [1]:
try:
    from dandi_utils import dandi_download_open
except:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .

c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os
import suite2p
import numpy as np
from tifffile import imsave

### Downloading Ophys NWB Files
Here you can download several files for a subject that you can run nway matching on. The pipeline can take in any number of input *sessions*, however, the input ophys data should be from the same imaging plane of the same subject. To specify your own file to use, set `dandiset_id` to be the dandiset id of the files you're interested in. Also set `input_dandi_filepaths` to be a list of the filepaths on dandi of each file you're interested in providing to **Nway Matching**. When accessing an embargoed dataset, set `dandi_api_key` to be your DANDI API key.

In [3]:
dandiset_id = "000336"
dandi_filepath = "sub-634402/sub-634402_ses-1209063020-acq-1209359211raw_ophys.nwb"
dandi_api_key = os.environ["DANDI_API_KEY"]

In [4]:
io = dandi_download_open(dandiset_id, dandi_filepath, "./", dandi_api_key=dandi_api_key)
# from pynwb import NWBHDF5IO
# io = NWBHDFIO("./sub-661749_ses-1254161187-acq-1254305759raw_ophys.nwb", load_namespaces=True)
nwb = io.read()

File already exists
Opening file


c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.3.0 because version 2.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


### Preparing Suite2P Input

In [5]:
movie = np.array(nwb.acquisition["raw_suite2p_motion_corrected"].data).astype(np.uint16)
movie

array([[[ 540,  499,  485, ...,  516,  534,  503],
        [ 520, 1422,  507, ...,  523,  509,  549],
        [ 760,  574,  488, ...,  808,  522, 1156],
        ...,
        [1198,  989,  522, ...,  677,  512,  459],
        [ 482,  498,  871, ..., 1739,  615,  506],
        [1018,  506,  498, ...,  506,  539,  506]],

       [[1347, 1978, 1033, ...,  500,  536, 2126],
        [1136,  521,  712, ...,  492,  858, 2157],
        [ 441,  540,  995, ...,  447,  502, 2129],
        ...,
        [ 505, 1366,  480, ...,  863,  507, 2800],
        [ 494,  753,  831, ..., 1173,  461,  497],
        [ 442,  565,  551, ...,  512,  532, 1989]],

       [[ 965,  550, 1295, ..., 1420,  506,  538],
        [ 539,  503, 1820, ..., 1570, 1455,  554],
        [ 502,  506,  517, ...,  453,  528,  867],
        ...,
        [1058,  515,  548, ..., 1085,  628,  481],
        [ 474,  547,  666, ...,  530,  495, 1378],
        [1064,  475,  501, ...,  532, 1646,  519]],

       ...,

       [[ 499,  682,  48

In [6]:
imsave("movie.tiff", movie)
# image_file = image_file.convert("L")
# image_file.save(f"./2p_movie.tif")

C:\Users\carter.peene\AppData\Local\Temp\ipykernel_16040\2967565980.py:1: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave("movie.tiff", movie)


In [7]:
results_folder = "./results/"
scratch_folder = "./results/"
input_movie_path = "./movie"
# sampling_rate = nwb.acquisition["raw_suite2p_motion_corrected"].rate
sampling_rate = 100
suite2p_threshold_scaling = 1

### Running Suite2P

In [8]:
ops = suite2p.default_ops()
ops['threshold_scaling'] = suite2p_threshold_scaling
ops['fs'] = float(sampling_rate) # sampling rate of recording, determines binning for cell detection
ops['tau'] = 0.7 # timescale of gcamp to use for deconvolution
ops['do_registration'] = 0 # data was already registered
ops['save_NWB'] = 1
ops['save_folder'] = results_folder
ops['fast_disk'] = scratch_folder

db = {
'data_path': ["./"]
}
output_ops = suite2p.run_s2p(ops=ops, db=db)

{'data_path': ['./']}


tif
** Found 1 tifs - converting to binary **
NOTE: ScanImageTiffReader not installed, using tifffile
2000 frames of binary, time 5.63 sec.
4000 frames of binary, time 10.13 sec.
6000 frames of binary, time 14.78 sec.
8000 frames of binary, time 19.30 sec.
10000 frames of binary, time 23.72 sec.
12000 frames of binary, time 28.17 sec.
14000 frames of binary, time 32.47 sec.
16000 frames of binary, time 36.84 sec.
18000 frames of binary, time 41.32 sec.
20000 frames of binary, time 45.77 sec.
22000 frames of binary, time 50.03 sec.
24000 frames of binary, time 54.46 sec.
26000 frames of binary, time 59.02 sec.
28000 frames of binary, time 63.48 sec.
30000 frames of binary, time 68.11 sec.
32000 frames of binary, time 72.60 sec.
34000 frames of binary, time 77.02 sec.
36000 frames of binary, time 81.69 sec.
38000 frames of binary, time 86.18 sec.
40000 frames of binary, time 90.57 sec.
42000 frames of binary, time 95.12 sec.
44000 frames of binary, time 99.77 sec.
time 101.58 sec. Wrote 

### Suite2P Output

In [9]:
flr_trace = np.load("./results/plane0/F.npy")
flr_trace.shape

(61, 44201)

- download NWB
- save raw movie
- determine inputs
- run suite2p pipeline
- parse output
- compare to segmentation in NWB